In [1]:
import sqlite3
import os

if not os.path.exists('mydatabase.db'):
    open('mydatabase.db', 'w').close()
conn = sqlite3.connect('mydatabase.db')
c = conn.cursor()

c.execute('''CREATE TABLE emotions
             (Date TEXT, 
             Male_Angry INTEGER, 
             Male_Fear INTEGER, 
             Male_Happy INTEGER, 
             Male_Neutral INTEGER, 
             Male_Sad INTEGER, 
             Male_Surprise INTEGER, 
             Male_Disgust INTEGER,
             Female_Angry INTEGER, 
             Female_Fear INTEGER, 
             Female_Happy INTEGER, 
             Female_Neutral INTEGER, 
             Female_Sad INTEGER, 
             Female_Surprise INTEGER, 
             Female_Disgust INTEGER)''')

conn.commit()
conn.close()


OperationalError: table emotions already exists

In [ ]:
from datetime import date
import sqlite3

def insert_emotions(emotions_dict):
    conn = sqlite3.connect('mydatabase.db')
    c = conn.cursor()
    
    today = date.today().strftime('%Y-%m-%d')
    
    values = [today] + list(emotions_dict.values())
    placeholders = ','.join(['?' for _ in range(len(values))])
    c.execute(f'INSERT INTO emotions VALUES ({placeholders})', values)
    
    conn.commit()
    conn.close()

In [2]:
import tensorflow
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
import os
import traceback
import sys

model1 = load_model(r'newacccc.h5')
model2 = load_model(r'prese.h5')
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprise"}
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
eg_dict = {'Male_Angry': 0, 'Male_Fear': 0, 'Male_Happy': 0, 'Male_Neutral': 0, 'Male_Sad': 0, 'Male_Surprise': 0, 'Male_Disgust': 0, 'Female_Angry': 0, 'Female_Fear': 0, 'Female_Happy': 0, 'Female_Neutral': 0, 'Female_Sad': 0, 'Female_Surprise': 0, 'Female_Disgust': 0}
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        for (x, y, w, h) in faces:
            face = frame[y:y+h , x:x+w]
            img = cv2.resize(face, (96, 96))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB color space
            img = img.astype('float32') / 255.0
            img = np.expand_dims(img, axis=0)
            pred = model2.predict(img)[0][0]
            if pred < 0.5:
                gender = 'Female'
            else:
                gender = 'Male'
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model1.predict(cropped_img)
            emotion =emotion_dict[int(np.argmax(prediction))]
            key = gender+'_'+emotion
            eg_dict[key]+=1
            cv2.putText(frame, f'{gender} , {emotion}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.imshow('Gender Detection', frame)

        # Exit the loop if the user presses the 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            insert_emotions(eg_dict)
            cap.release()
            cv2.destroyAllWindows()
            break
except :
    insert_emotions(eg_dict)
    cap.release()
    cv2.destroyAllWindows()  
    traceback.print_exception(*sys.exc_info())


1/1 [==============================] - 0s 31ms/step


NameError: name 'insert_emotions' is not defined

In [6]:
cap.release()
cv2.destroyAllWindows()

In [6]:
print(r)

C:/Users/DELL/img
